# imports

In [ ]:
from module import *

# system init
This is a generic spring damper setup. Very simple and linear, with two states and one input. The state euation is the force balance of a spring damper.

In [ ]:
spring_system = SpringSystem(set_seed=0)

# surrogate generator init
This class is designed to be totally generic, i.e., it can take any do-mpc model, or at least that is the idea.

In [ ]:
su = SurrogateCreator(set_seed=0)

# random data

Here we generate sampled with one random initial point and random inputs. Then the data and the data is split randomly to feed different parts of the algorithm.
There is another alternate algorithm which generates data by chasing random setpoints with the help of an MPC controller.

In [ ]:
su.random_input_sampler(system = spring_system, n_samples=1000)
#su.data_splitter(order=3, cqr_calibration=0.001, test=0.001, narx_train=0.998, cqr_train=0.998)
su.data_splitter(order=3)

## data visualisation

In [ ]:
su.visualize2d_data()

In [ ]:
su.visualize_data()

# NARX model

In [ ]:
su.narx_trainer(hidden_layers=[1], batch_size=320,
          learning_rate=0.1, epochs= 1000, scheduler_flag=True)
su.plot_narx_training_history_plotly()

# quantile regression
Here qunatile regression is done to bound the errors with confidence values

In [ ]:
su.train_individual_qr(alpha=0.2, hidden_layers=[10], batch_size=320)
su.plot_qr_training_history_plotly()

This section visualises the quantile regression on the calibration data which the regressors has yet not seen.

In [ ]:
su.plot_qr_error()

# conformalisation
The quantile regressors are conformalised with calibration data.

In [ ]:
su.conform_qr()

This plot is made against test data which till now is untouched.

In [ ]:
su.plot_cqr_error_plotly()

# simulation

In [ ]:
su.run_simulation(iter=50, n_horizon=10, r=0.1)
su.plot_simulation()